In [1]:
import pandas as pd
import numpy as np
import os
import pandas_gbq
from google.cloud import bigquery
import glob
import openpyxl

In [2]:
os.chdir('G:\\\\Drives compartilhados\\\\República.org\\\\4. Equipes\\\\Dados e Comunicação\\\\DADOS\\\\415 - Repositório de Dados\\\\Repositório Local')


Chamando as bases

In [3]:
df18 = pd.read_excel('ESTADIC//Base_ESTADIC_2018.xlsx', sheet_name='Educação', usecols=['Cod Uf','EEDU491','EEDU49212','EEDU49211','EEDU49213'])
df18

,Cod Uf,EEDU491,EEDU49211,EEDU49212,EEDU49213
0,11,Não,Não,Não,Sim
1,12,Não,Não,Sim,Sim
2,13,Não,Sim,Não,Não
3,14,Não,Sim,Não,Não
4,15,Não,Sim,Sim,Sim
5,16,Não,Sim,Sim,Sim
6,17,Não,Sim,Não,Sim
7,21,Não,Sim,Não,Não
8,22,Não,Não,Sim,Sim
9,23,Não,Não,Não,Sim


In [4]:
df21 = pd.read_excel('ESTADIC//Base_ESTADIC_2021.xlsx', sheet_name='Educação', usecols=['Cod UF','EEDU491','EEDU49212','EEDU49211','EEDU49213'])
df21

,Cod UF,EEDU491,EEDU49211,EEDU49212,EEDU49213
0,11,Não,Sim,Sim,Não
1,12,Não,Não,Sim,Não
2,13,Não,Não,Não,Não
3,14,Não,Não,Não,Não
4,15,Não,Sim,Sim,Não
5,16,Não,Sim,Não,Não
6,17,Não,Sim,Não,Não
7,21,Não,Sim,Sim,Sim
8,22,Não,Sim,Sim,Sim
9,23,Não,Sim,Sim,Sim


In [5]:
uf = pd.read_excel('ESTADIC//Base_ESTADIC_2018.xlsx', sheet_name = 'Variáveis externas', usecols=[1,2,3])
uf

,COD UF,UF,NOME UF
0,11,RO,RONDÔNIA
1,12,AC,ACRE
2,13,AM,AMAZONAS
3,14,RR,RORAIMA
4,15,PA,PARÁ
5,16,AP,AMAPÁ
6,17,TO,TOCANTINS
7,21,MA,MARANHÃO
8,22,PI,PIAUÍ
9,23,CE,CEARÁ


Renomeando as colunas

In [6]:
df18= df18.rename(columns={'Cod Uf':'cod_uf',
                        'EEDU491':'nomeacao_concurso_publico',
                        'EEDU49212':'nomeacao_eleicao',
                        'EEDU49211' :'nomeacao_indicacao',
                        'EEDU49213':'nomeacao_processo_seletivo'}) 

In [7]:
df21= df21.rename(columns={'Cod UF':'cod_uf',
                        'EEDU491':'nomeacao_concurso_publico',
                        'EEDU49212':'nomeacao_eleicao',
                        'EEDU49211' :'nomeacao_indicacao',
                        'EEDU49213':'nomeacao_processo_seletivo'}) 

In [8]:
##adicionando a coluna de 'ano' em cada um dos dfs
df18['ano']=2018
df21['ano']=2021 

In [9]:
df = pd.concat([df18,df21]) #juntando os dataframes

In [10]:
x= uf.pivot_table(columns=('COD UF', 'UF', 'NOME UF'), aggfunc='size')


In [11]:
uf = pd.DataFrame(x).reset_index()[['COD UF', 'UF', 'NOME UF']]

In [12]:
df = df.merge(uf, right_on='COD UF',left_on='cod_uf') #adicionando sigla e nome das UFs


In [13]:
df = df.drop(['COD UF'], axis=1) #eliminando coluna repetida

In [14]:
df= df.rename(columns={'UF':'sigla_uf',
                'NOME UF':'uf'})

In [21]:
df['nomeacao_indicacao']=np.where(df['nomeacao_indicacao']=='-','Sem dados',df['nomeacao_indicacao'])
df['nomeacao_eleicao']=np.where(df['nomeacao_eleicao']=='-','Sem dados',df['nomeacao_eleicao'])
df['nomeacao_processo_seletivo']=np.where(df['nomeacao_processo_seletivo']=='-','Sem dados',df['nomeacao_processo_seletivo'])


In [30]:
df.columns

Index(['ano', 'cod_uf', 'sigla_uf', 'uf', 'nomeacao_concurso_publico',
       'nomeacao_indicacao', 'nomeacao_eleicao', 'nomeacao_processo_seletivo'],
      dtype='object')

In [39]:
df['nomeacao_processo_seletivo'].unique()

array(['Sim', 'Não', 'Sem dados'], dtype=object)

In [25]:
df = df[['ano','cod_uf','sigla_uf','uf','nomeacao_concurso_publico', 'nomeacao_indicacao',
       'nomeacao_eleicao', 'nomeacao_processo_seletivo']]

In [26]:
df['uf'] = df['uf'].str.title()

Subindo para o GBQ

In [159]:
client = bigquery.Client()
dataset_ref = client.dataset('cargos_lideranca')

In [168]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 54 entries, 0 to 53
Data columns (total 8 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   ano                         54 non-null     int64 
 1   cod_uf                      54 non-null     int64 
 2   sigla_uf                    54 non-null     object
 3   uf                          54 non-null     object
 4   nomeacao_concurso_publico   54 non-null     object
 5   nomeacao_indicacao          54 non-null     object
 6   nomeacao_eleicao            54 non-null     object
 7   nomeacao_processo_seletivo  54 non-null     object
dtypes: int64(2), object(6)
memory usage: 3.8+ KB


In [169]:
schema=[bigquery.SchemaField('ano','INTEGER',description='Ano da apuração daquele dado'),
        bigquery.SchemaField('cod_uf','INTEGER',description='Código do IBGE da UF'),
        bigquery.SchemaField('sigla_uf','STRING',description='Sigla da UF'),
        bigquery.SchemaField('uf','STRING',description='Nome da UF'),        
        bigquery.SchemaField('nomeacao_concurso_publico','STRING',description='Se a nomeação de diretor de escola ocorreu por concurso público '),
        bigquery.SchemaField('nomeacao_indicacao','STRING',description='Se a nomeação de diretor de escola ocorreu por indicação'),
        bigquery.SchemaField('nomeacao_eleicao','STRING',description='Se a nomeação de diretor de escola ocorreu por eleição'),
        bigquery.SchemaField('nomeacao_processo_seletivo','STRING',description='Se a nomeação de diretor de escola ocorreu por processo seletivo')

 ]


In [170]:
table_ref = dataset_ref.table('ESTADIC_nomeacao_diretor_escola')
job_config = bigquery.LoadJobConfig(schema=schema)
job = client.load_table_from_dataframe(df,table_ref, job_config=job_config)
job.result() 

LoadJob<project=repositoriodedadosgpsp, location=US, id=3b6d8fa4-53bb-4565-a490-2668176826f1>